In [1]:
import os
import numpy as np
import json
from pathlib import Path
from keras.models import Model
from tensorflow import keras
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, AveragePooling2D,Conv2D
)
from keras.layers import ( Input, Activation, Dense, Flatten )
#from keras.layers.convolutional import ( Conv2D, MaxPooling2D , AveragePooling2D) //DMR se utilizo tensorflow.keras.layers
from keras.layers.merge import add
#from keras.layers.normalization import BatchNormalization //DMR se utilizo tensorflow.keras.layers
from keras.regularizers import l2
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping



ModuleNotFoundError: No module named 'keras'

In [3]:
path = os.path.abspath(os.getcwd())+'\\resultados\\'

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

train_data = np.load(path+'train_data.npy')
train_labels = np.load(path+'train_labels.npy')
test_data = np.load(path+'test_data.npy')
test_labels = np.load(path+'test_labels.npy')
labels = np.load(path+'labels.npy')

# restore np.load for future normal usage
np.load = np_load_old

In [4]:
print( "train data shape: ",  train_data.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_data.shape )
print( "test_labels.shape: ", test_labels.shape )

train data shape:  (100000, 64, 64, 3)
train label shape:  (100000, 200)
test data shape:  (10000, 64, 64, 3)
test_labels.shape:  (10000, 200)


In [5]:
def shuffle_data(train_data, train_labels ):
    size = len(train_data)
    train_idx = np.arange(size)
    np.random.shuffle(train_idx)

    return train_data[train_idx], train_labels[train_idx]
  
train_data, train_labels = shuffle_data(train_data, train_labels)

In [6]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet50_tiny_ImageNet.csv')

batch_size = 100
nb_classes = 200
nb_epoch = 10

# input image dimensions
img_rows, img_cols = 64, 64
# The images are RGB
img_channels = 3

input_shape = (img_rows, img_cols, img_channels)

# The data, shuffled and split between train and test sets:
X_train = train_data
Y_train = train_labels
X_test = test_data
Y_test = test_labels
del train_data
del train_labels
del test_data
del test_labels
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.


In [7]:
# Se utilia un modelo de KERAS el resnet50
resnet50v2 = keras.applications.ResNet50V2(include_top=True, classes=nb_classes, input_shape=input_shape, weights=None)
#resnet50v2.summary()

resnet50v2.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['acc'])


In [8]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
          featurewise_center=False,           # set input mean to 0 over the dataset
          samplewise_center=False,            # set each sample mean to 0
          featurewise_std_normalization=False,# divide inputs by std of the dataset
          samplewise_std_normalization=False, # divide each input by its std
          zca_whitening=False,                # apply ZCA whitening
          rotation_range=0,                   # randomly rotate images in the range (degrees, 0 to 180)
          width_shift_range=0.1,              # randomly shift images horizontally (fraction of total width)
          height_shift_range=0.1,             # randomly shift images vertically (fraction of total height)
          horizontal_flip=True,               # randomly flip images
          vertical_flip=False )               # randomly flip images

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit( X_train )

Using real-time data augmentation.


In [9]:
nb_epoch = 10 # epoch 1-10
# Fit the model on the batches generated by datagen.flow().
resnet50v2.fit_generator( datagen.flow(X_train, Y_train, batch_size=batch_size),
                     steps_per_epoch=X_train.shape[0] // batch_size,
                     validation_data=(X_test, Y_test),
                     epochs=nb_epoch, verbose=1,
                     callbacks=[lr_reducer, early_stopper, csv_logger] )
                     
# El modelo es entrenado con el dataset cifar10
with open("history_resnet50v2.json", "w") as ff:
    json.dump(resnet50v2.history, ff)

C:\Users\DAVID\AppData\Local\Temp/ipykernel_26492/1204747456.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  resnet50v2.fit_generator( datagen.flow(X_train, Y_train, batch_size=batch_size),


Epoch 1/10
 23/200 [==>...........................] - ETA: 1:01:22 - loss: 5.3920 - acc: 0.0140

KeyboardInterrupt: 